In [113]:
import pandas as pd
import numpy as np

RATINGS_EXTRACTING_PATTERN = r"(\d+\.?\d*)"

In [114]:
data = pd.read_csv("amazon_clothing_product_data.csv", index_col=0);

In [115]:
## 1. Handling Missing Values

# There are 0 missing values in the Product Name column.
data["Product Name"].isna().sum()

# Filling Item model number with default value "DEFAULT-160303"
data["Item model number"].fillna("DEFAULT-160303", inplace=True)

# There are 0 missing values in the ASIN column.
data["ASIN"].isna().sum()

# There are 0 missing values in the Department column.
data["Department"].isna().sum()

# There are 0 missing values in the Date First Available column.
data["Date First Available"].isna().sum()

# Fill the missing values in the Product Dimensions column with default value "17.5 x 14.5 x 1.5 cm; 330 g".
data["Product Dimensions"].fillna("17.5 x 14.5 x 1.5 cm; 330 g", inplace=True)

# Filling Item model number with default value "S, M, L, XL, 2XL"
data["Sizes Available"].fillna("S, M, L, XL, 2XL", inplace=True)

# There is too much missing values in the Colours Available column, So we will drop this column.
data.drop(["Colours Available"], axis=1,inplace=True)

# Filling Collar style with default value "Collarless"
data["Collar style"].fillna("Collarless", inplace=True)

# Filling Fit type with default value "Regular Fit"
data["Fit type"].fillna("Regular Fit", inplace=True)

# Length column is not very important, So we will drop this column.
data.drop(["Length"], axis=1, inplace=True)

# Material composition column is not very important, So we will drop this column.
data.drop(["Material composition"], axis=1, inplace=True)

# Filling Country of Origin with default value "India"
data["Country of Origin"].fillna("India", inplace=True)

# Filling Manufacturer with default value "Not Available"
data["Manufacturer"].fillna("Not Available", inplace=True)

# Packer and Importer columns are not very important, So we will drop these columns.
data.drop(["Packer", "Importer"], axis=1, inplace=True);

# Filling Item Weight with default value "250 g"
data["Item Weight"].fillna("250 g", inplace=True)

# There are 0 missing values in the Price column.
data["Discount (in %)"].isna().sum()

# There are 0 missing values in the Price column.
data["Price (in Rs.)"].isna().sum()

# Filling the Nan values in the Ratings column with default value "4.5" and
# Extracting the actual ratings stars from the Ratings column.
data["Ratings"].fillna("4.5", inplace=True)
data["Ratings"] = data["Ratings"].str.extract(RATINGS_EXTRACTING_PATTERN)

# Product Description column is not very important, So we will drop this column.
data.drop(["Product Description"], axis=1, inplace=True)

# Saving the cleaned dataset.
data.to_csv("cleaned_amazon_clothing_product_data.csv");

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1099 entries, 0 to 1098
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Product Name          1099 non-null   object
 1   Item model number     1099 non-null   object
 2   ASIN                  1099 non-null   object
 3   Department            1099 non-null   object
 4   Date First Available  1099 non-null   object
 5   Product Dimensions    1099 non-null   object
 6   Sizes Available       1099 non-null   object
 7   Collar style          1099 non-null   object
 8   Fit type              1099 non-null   object
 9   Country of Origin     1099 non-null   object
 10  Manufacturer          1099 non-null   object
 11  Item Weight           1099 non-null   object
 12  Discount (in %)       1099 non-null   int64 
 13  Price (in Rs.)        1099 non-null   int64 
 14  Ratings               1099 non-null   object
dtypes: int64(2), object(13)
memory usage: 137.4

In [116]:
# 2. Extract the numeric values from column where data is present in the form of object and Standardizing Data Types.

# For Product Dimensions Column.
def extract_dimensions(value):
    """Extracts dimensions (length, width, height) from a string in cm format."""

    try:
        # Split the string, handling potential semicolons for weight separation
        parts = value.split(";")
        dimensions_str = parts[0].strip()  # Get the part containing dimensions

        # Extract dimensions using regular expression (more robust)
        import re

        matches = re.findall(
            r"(\d+\.\d+|\d+) ?x ?(\d+\.\d+|\d+) ?x ?(\d+\.\d+|\d+)", dimensions_str
        )
        if matches:
            dimensions = [float(x) for x in matches[0]]
            return dimensions
        else:
            return None

    except (ValueError, AttributeError):
        # Handle potential errors during parsing
        print(f"Error parsing dimensions for value: {value}")
        return None

dimensions = data["Product Dimensions"].apply(extract_dimensions)

# Check if dimensions extraction was successful (all non-None values)
if all(dim is not None for dim in dimensions):
    # Create new columns for length, width, height (assuming cm units)
    data.insert(9, "length_cm", [dim[0] for dim in dimensions])
    data.insert(10, "width_cm", [dim[1] for dim in dimensions])
    data.insert(11, "height_cm", [dim[2] for dim in dimensions])

    # Drop the original column
    data.drop(["Product Dimensions"], axis=1, inplace=True)


# For Item Weight Column.
# Extract weight from the Item Weight column, convert to float and drop the Item Weight column.
data.insert(11, "weight_g", data["Item Weight"].str.extract(r"(\d+)"))
data["weight_g"] = data["weight_g"].astype(float);
data.drop(["Item Weight"], axis=1, inplace=True);

# Convert the Discount column to float and drop the Discount column.
data.insert(14, "discount_%", data["Discount (in %)"].astype(float))
data.drop(["Discount (in %)"], axis=1, inplace=True);

# Convert the Price column to float and drop the Price column.
data.insert(15, "price_rs", data["Price (in Rs.)"].astype(float))
data.drop(["Price (in Rs.)"], axis=1, inplace=True);

# Saving the cleaned dataset.
data.to_csv("cleaned_amazon_clothing_product_data.csv")

In [117]:
# 3. Renaming all columns.

data.rename(
    columns={
        "Product Name": "product_name",
        "Item model number": "item_model_number",
        "ASIN": "asin",
        "Department": "department",
        "Date First Available": "date_first_available",
        "Sizes Available": "sizes_available",
        "Collar style": "collar_style",
        "Fit type": "fit_type",
        "Country of Origin": "country_of_origin",
        "Manufacturer": "manufacturer",
        "Ratings": "ratings",
    },
    inplace=True
)


In [118]:
# 4. Saving the cleaned dataset.
data.to_csv("cleaned_amazon_clothing_product_data.csv")

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1099 entries, 0 to 1098
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_name          1099 non-null   object 
 1   item_model_number     1099 non-null   object 
 2   asin                  1099 non-null   object 
 3   department            1099 non-null   object 
 4   date_first_available  1099 non-null   object 
 5   sizes_available       1099 non-null   object 
 6   collar_style          1099 non-null   object 
 7   fit_type              1099 non-null   object 
 8   length_cm             1099 non-null   float64
 9   width_cm              1099 non-null   float64
 10  height_cm             1099 non-null   float64
 11  weight_g              1099 non-null   float64
 12  country_of_origin     1099 non-null   object 
 13  manufacturer          1099 non-null   object 
 14  discount_%            1099 non-null   float64
 15  price_rs              1099